<a href="https://colab.research.google.com/github/Frodo-Swaggins/COMP702PROJECT/blob/main/COMP719_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code Reference: https://medium.com/walmartglobaltech/automating-shift-scheduling-with-linear-programming-fe1720f13620

Import Libraries

In [1]:
!pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.2 MB 5.4 MB/s 


In [2]:
from pulp import *
import pandas as pd

In [3]:
def setSensitivity():
  global employees, days, min_employeepershift_val, max_consecuvtive_days_off, level, max_consecutive_days_work
  print('Enter number of employees that are currently hired')
  num_of_employees = int(input())
  print('Enter number of days in a schedule cycle')
  num_of_days = int(input())
  print('Enter the minimum number of employees require per shift')
  min_employees_per_shift = int(input())
  print('Enter the maximum number of consecutive days an employee can be on leave for')
  max_consecutive_days_leave= int(input())
  print('Enter level')
  lev = int(input())
  print('Enter the maximum number of consecutive days an employee can work for')
  max_consecutive_days_on = int(input())
  
  #Minimum Inputs - Resource list 
  employees = num_of_employees
  days = num_of_days
  #minimum number of nurses needed for a= single shift
  min_employeepershift_val = min_employees_per_shift
  #max consecutive days a nurse can work
  max_consecutive_days_off = 2#max_consecutive_days_leave#Can be lowered but not necessary
  #as level increases by one the amount of overall resources is doubles
  level = lev
  #max number of days an employee can work consecutively
  max_consecutive_days_work = max_consecutive_days_on

In [4]:
def addConstraintOne():
  #constraint 1
  global model, var_M, var_E, var_L, employees, days
  # each employee can work at most one shift per day or be on leave on that day
  for i in range(days):
      for j in range(employees):
          numShifts = None
          numShifts += var_M[i][j] + var_E[i][j] + var_L[i][j]
          model += numShifts == 1

In [5]:
def addConstraintTwo():
  #constraint 2
  #we should ensure the minimum required staff for a sgift is met
  global model, var_M, var_E, min_employeepershift_val, employees, days
  for i in range(0, days):
      numworkingmorningshift = None
      numworkingeveningshift = None    
      for j in range(employees):
          numworkingmorningshift += var_M[i][j]
          numworkingeveningshift += var_E[i][j]

      model += numworkingmorningshift == min_employeepershift_val#== -> >=
      model += numworkingeveningshift == min_employeepershift_val
  

In [6]:
def addConstraintThree():
  #constraint 3
  global model, var_M, var_E, employees, days
  #employee should work the same shift the next the next day
  for j in range(employees):
      for i in range(days - 1):
        constraint = None
        #on 2 consecutive day a specific employee should be working the same shift
        constraint += var_M[i][j] + var_E[i+1][j]
        model += constraint <= 1

      for j in range(employees):
          for i in range(days - 1):
              if ((i + 1) % days != 0) and ((i + 2) % days != 0):
                constraint = None
                constraint += var_E[i][j] + var_M[i+1][j]
                model += constraint <= 1

In [7]:
def addConstraintFour():
  #constraint 4
  global model, var_M, employees, days
  #every 2 weeks nurses have their shifts switched
  for j in range(employees):
      for i in range(0, days - (days*2), 2):
          constraint = None
          constraint += var_M[i][j] + var_M[i+days][j] + var_M[i+(days*2)][j]
          model += constraint <= 2

In [8]:
def addConstraintFive():
  #constraint 5
  global model, var_M, var_E, max_consecutive_days_work, employees, days
  #a nurse can work at most 5 days in a row
  for j in range(employees):
      for i in range(0, days- (days-1)):
        consecutiveDaysWorked = None
        for w in range(days):
            consecutiveDaysWorked += var_M[i+w][j] + var_E[i+w][j]
        model += consecutiveDaysWorked <= max_consecutive_days_work

In [9]:
def addConstraintSix():
  #constraint 6
  global model, max_consecutive_days_off, var_L, employees, days
  #nurses can have at most a certia number of days of consecutive days off
  #max_consecutive_leave= 2
  for j in range(employees):
      for i in range(0, days-2):
        consecutiveDaysLeave = None
        consecutiveDaysLeave += var_L[i][j] - var_L[i+1][j] + var_L[i+2][j] 
        model += consecutiveDaysLeave <= max_consecutive_days_off

In [10]:
def solveModel():
  global model
  # Solve problem. Whole tasks is passes pulp framework!
  #status = problem.solve(problem)# If pulp can find optimal values which satisfies conditions, print “Optimal”
  status = model.solve()
  print('Status: ', LpStatus[status])
  #Objective function value with all constraints satisfied
  print('Objective = ',value(model.objective))

In [22]:
# Global variables

#Minimum Inputs - Resource list 
employees = 0
days = 0
#minimum number of nurses needed for a= single shift
min_employeepershift_val = 0
#max consecutive days a nurse can work
max_consecutive_days_off = 0#Can be lowered but not necessary
#as level increases by one the amount of overall resources is doubles
level = 0
#max number of days an employee can work consecutively
max_consecutive_days_work = 0

var_M = {}
var_E = {}
var_L = {}

model = LpProblem('Shift_Scheduling', LpMaximize)

def run():
  setSensitivity()
  print('employees-',employees,'days-', days, 'emp per shift-', min_employeepershift_val,'max days leave-',max_consecutive_days_off,'level-',level,'max days of work in a row',max_consecutive_days_work)
  
  global var_M
  global var_E
  global var_L
  global model

  #Add decision variables

  #var_M = pulp.LpVariable.dicts('WorkingMorningShift', (range(days), range(employees)), 0, 1, 'Binary')
  var_M = pulp.LpVariable.dicts('M',(range(days),range(employees)),lowBound=0,cat='Binary')
  #var_E = pulp.LpVariable.dicts('WorkingEveningShift', (range(days), range(employees)), 0, 1, 'Binary')
  var_E= pulp.LpVariable.dicts('E',(range(days),range(employees)),lowBound=0,cat='Binary')
  #var_L = pulp.LpVariable.dicts('WorkingLeaveShift', (range(days), range(employees)), 0, 1, 'Binary')
  var_L= pulp.LpVariable.dicts('L',(range(days),range(employees)),lowBound=0,cat='Binary')

  #Objective function definition
  objectivefunction = None
  #level = 1
  for i in range(days):
      for j in range(employees):
          objectivefunction += (level * var_M[i][j]) + (level * var_E[i][j]) # + var_L[i][j]
  model += objectivefunction
  
  #add contraints to model
  addConstraintOne()
  addConstraintTwo()
  addConstraintThree()
  addConstraintFour()
  addConstraintFive()
  addConstraintSix()
  
  #Solve model if solvable
  solveModel()

  

In [23]:
run()

Enter number of employees that are currently hired
8
Enter number of days in a schedule cycle
5
Enter the minimum number of employees require per shift
4
Enter the maximum number of consecutive days an employee can be on leave for
2
Enter level
1
Enter the maximum number of consecutive days an employee can work for
5
employees- 8 days- 5 emp per shift- 4 max days leave- 0 level- 1 max days of work in a row 5
Status:  Optimal
Objective =  40.0


Sensitivity Analysis

In [25]:

o = [{'Name':name,'Constraint':c,'shadow price':c.pi,'slack': c.slack} for name, c in model.constraints.items()]

print(pd.DataFrame(o))

      Name                       Constraint  shadow price  slack
0      _C1   {M_0_0: 1, E_0_0: 1, L_0_0: 1}          -0.0   -0.0
1      _C2   {M_0_1: 1, E_0_1: 1, L_0_1: 1}          -0.0   -0.0
2      _C3   {M_0_2: 1, E_0_2: 1, L_0_2: 1}          -0.0   -0.0
3      _C4   {M_0_3: 1, E_0_3: 1, L_0_3: 1}          -0.0   -0.0
4      _C5   {M_0_4: 1, E_0_4: 1, L_0_4: 1}          -0.0   -0.0
..     ...                              ...           ...    ...
301  _C302  {L_1_6: 1, L_2_6: -1, L_3_6: 1}          -0.0   -0.0
302  _C303  {L_2_6: 1, L_3_6: -1, L_4_6: 1}          -0.0   -0.0
303  _C304  {L_0_7: 1, L_1_7: -1, L_2_7: 1}          -0.0   -0.0
304  _C305  {L_1_7: 1, L_2_7: -1, L_3_7: 1}          -0.0   -0.0
305  _C306  {L_2_7: 1, L_3_7: -1, L_4_7: 1}          -0.0   -0.0

[306 rows x 4 columns]


In [ ]:
#print(model)

In [ ]:
# variables = []
# for v in model.variables():
#   variables.append(v.varValue)
#   print(v.name,"=",v.varValue)


In [ ]:
# dayList = ['ON MONDAY','ON TUESDAY','ON WEDNESDAY','ON THURS','ON FRIDAY','ON SATURDAY','ON SUNDAY']
# shiftList = ['WORKING EVENING SHIFT',' WORKING MORNINGSHIFT','ON LEAVE']
# print(len(variables))
# print()
# c =0
# cs = 0
# i = 0
# for s in range (days*employees):
#   if(cs>2):
#     cs = 0
#     print(shiftList[cs])
#   else:
#     print(shiftList[cs])
#   for j in range(days):
#     print(dayList[j],' :')
#     for i in range(employees):
#       if(variables[c]==1.0 and (c < len(variables))):
#         print("Employee ", (i+1))
#         print()
#       if(i == employees):
#         i =0
#       c = c+1
#   cs = cs +1